In [ ]:
# Install required libraries
# !pip install plotly
# !pip install ipywidgets

import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, HTML
import numpy as np

In [ ]:
# Define products data structure
products = {
    "Foundation": [
        ("LES BEIGES", 70),
        ("ULTRA LE TEINT PRODUCT 1", 70),
        ("N°1 DE CHANEL REVITALIZING FOUNDATION", 70),
        ("LES BEIGES (Tint)", 70),
        ("ULTRA LE TEINT PRODUCT 2", 70),
        ("VITALUMIÈRE AQUA", 70),
        ("BOY DE CHANEL", 70),
        ("ULTRA LE TEINT VELVET", 70)
    ],
    "Bronzer": [("Bronzing Powder", 70)],
    "Highlighter": [("Highlighting Powder", 70)],
    "Blush": [("Blush Palette", 70)],
    "Powder": [("Compact Powder", 70)],
    "Primer": [("Makeup Primer", 70)],
    "Concealer": [("Concealer Stick", 70)],
    "Brushes and Accessories": [("Makeup Brush Set", 70)],
    "Mascara": [("Volumizing Mascara", 70)],
    "Eyeshadow": [("Eyeshadow Palette", 70)],
    "Eyeliner": [("Liquid Eyeliner", 70)],
    "Brow": [("Brow Pencil", 70)],
    "Lipstick": [("Classic Lipstick", 70)],
    "Liquid Lipsticks": [("Matte Liquid Lipstick", 70)],
    "Lipgloss": [("Shiny Lipgloss", 70)],
    "Lip Balm": [("Hydrating Lip Balm", 70)],
    "Lip liner": [("Lip Contour Pencil", 70)],
    "Fragrance": [("Signature Perfume", 70)],
    "Cleansers & Makeup Removers": [("Makeup Remover Wipes", 70)],
    "Serums": [("Facial Serum", 70)],
    "Moisturizers": [("Daily Moisturizer", 70)],
    "Sun Protection": [("SPF Sunscreen", 70)],
    "Masks": [("Face Mask", 70)],
    "Body care": [("Body Lotion", 70)]
}

# Create detailed inventory data function
def create_inventory_df(products):
    inventory_data = []
    for category, items in products.items():
        for product_name, initial_stock in items:
            inventory_data.append({
                'Category': category,
                'Product': product_name,
                'Initial_Stock': initial_stock,
                'Used': 0,  # Initialize used units to 0
            })

    df = pd.DataFrame(inventory_data)
    df['Remaining'] = df['Initial_Stock'] - df['Used']
    return df

# Create dashboard function
def create_dashboard(df):
    # Create figure with subplots
    fig = make_subplots(
        rows=2, cols=2,
        specs=[[{"type": "indicator"}, {"type": "indicator"}],
               [{"type": "bar", "colspan": 2}, None]],
        subplot_titles=("Total Products", "Total Remaining Stock", "Inventory Status by Product")
    )

    # Add indicators
    fig.add_trace(
        go.Indicator(
            mode="number",
            value=df['Initial_Stock'].sum(),
            title={"text": "Total Initial Stock"}
        ),
        row=1, col=1
    )

    fig.add_trace(
        go.Indicator(
            mode="number",
            value=df['Remaining'].sum(),
            title={"text": "Total Remaining Stock"}
        ),
        row=1, col=2
    )

    # Add bar chart for products
    fig.add_trace(
        go.Bar(
            name='Initial Stock',
            x=df['Product'],
            y=df['Initial_Stock'],
            text=df['Category'],
            hovertemplate="Product: %{x}<br>" +
                         "Category: %{text}<br>" +
                         "Initial Stock: %{y}<br>"
        ),
        row=2, col=1
    )

    fig.add_trace(
        go.Bar(
            name='Remaining',
            x=df['Product'],
            y=df['Remaining'],
            text=df['Category'],
            hovertemplate="Product: %{x}<br>" +
                         "Category: %{text}<br>" +
                         "Remaining: %{y}<br>"
        ),
        row=2, col=1
    )

    # Update layout
    fig.update_layout(
        height=1000,
        showlegend=True,
        title_text="Product-Level Inventory Dashboard",
        barmode='group'
    )

    # Rotate x-axis labels for better readability
    fig.update_xaxes(tickangle=45)

    # Show low stock alerts
    low_stock = df[df['Remaining'] <= df['Initial_Stock'] * 0.2]  # Alert if less than 20% remaining
    if not low_stock.empty:
        print("\n⚠️ Low Stock Alerts:")
        for _, row in low_stock.iterrows():
            print(f"- {row['Product']} ({row['Category']}): Only {row['Remaining']} / {row['Initial_Stock']} units remaining")

    return fig

# Create DataFrame
df = create_inventory_df(products)

# Create and display dashboard
fig = create_dashboard(df)
fig.show()

# Interactive stock update function
def update_stock(product, units_used):
    df.loc[df['Product'] == product, 'Used'] = units_used
    df['Remaining'] = df['Initial_Stock'] - df['Used']
    fig = create_dashboard(df)
    fig.show()

# Create dropdown for products
product_dropdown = widgets.Dropdown(
    options=df['Product'].tolist(),
    description='Product:',
    style={'description_width': 'initial'}
)

# Create slider for stock adjustment
stock_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=df.loc[df['Product'] == product_dropdown.value, 'Initial_Stock'].values[0],
    description='Units Used:',
    style={'description_width': 'initial'}
)

# Create update button
update_button = widgets.Button(description='Update Stock')

def on_button_click(b):
    update_stock(product_dropdown.value, stock_slider.value)

update_button.on_click(on_button_click)

# Display controls
display(HTML("<h3>Stock Adjustment Controls</h3>"))
display(product_dropdown)
display(stock_slider)
display(update_button)

# Display summary statistics
print("\nInventory Summary by Category:")
category_summary = df.groupby('Category').agg({
    'Initial_Stock': 'sum',
    'Remaining': 'sum'
}).round(2)
display(category_summary)

Dropdown(description='Product:', options=('LES BEIGES', 'ULTRA LE TEINT PRODUCT 1', 'N°1 DE CHANEL REVITALIZIN…

IntSlider(value=0, description='Units Used:', max=70, style=SliderStyle(description_width='initial'))

Button(description='Update Stock', style=ButtonStyle())


Inventory Summary by Category:


,Initial_Stock,Remaining
Category,,
Blush,70,70
Body care,70,70
Bronzer,70,70
Brow,70,70
Brushes and Accessories,70,70
Cleansers & Makeup Removers,70,70
Concealer,70,70
Eyeliner,70,70
Eyeshadow,70,70
